In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector

In [2]:
x_data_generated, y_data_generated = make_classification(scale=1)


In [3]:
begin_data = cross_val_score(LogisticRegression(), x_data_generated, y_data_generated, scoring='accuracy').mean()
begin_data

0.86

In [16]:
corr = pd.DataFrame(x_data_generated)

In [17]:
corr.corr().style.background_gradient(cmap='YlOrRd')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.025475,0.062286,-0.081699,-0.178629,0.048923,0.093876,-0.185457,-0.111444,0.067126,0.056398,-0.002916,-0.050556,0.024519,0.143609,-0.090782,0.106146,0.047424,-0.048626,0.120150
1,0.025475,1.000000,0.106825,0.086365,-0.033671,0.014401,0.004032,0.072380,0.108443,0.113275,-0.131637,-0.035135,-0.011838,0.098443,-0.007472,-0.132777,0.011981,0.055997,0.147892,0.069401
2,0.062286,0.106825,1.000000,-0.076242,0.166683,0.051939,0.168375,0.191442,-0.139544,-0.083372,0.109578,-0.017654,-0.003696,0.943548,-0.157613,0.048524,0.045751,-0.021236,0.138138,0.596300
3,-0.081699,0.086365,-0.076242,1.000000,0.059328,-0.182961,0.100969,-0.141302,-0.080367,0.006003,-0.159875,-0.205753,0.097263,-0.039661,0.100361,-0.150547,-0.100832,-0.061874,-0.097402,-0.123690
4,-0.178629,-0.033671,0.166683,0.059328,1.000000,0.070802,0.055982,0.156709,-0.105847,-0.027738,-0.058889,0.047955,-0.018922,0.174940,-0.101210,0.145256,-0.045652,-0.185017,-0.051243,0.056578
5,0.048923,0.014401,0.051939,-0.182961,0.070802,1.000000,-0.109489,0.146327,0.037683,-0.033446,-0.040321,-0.124869,-0.119407,-0.006108,0.100874,-0.016648,0.168594,0.013805,0.001795,0.164544
6,0.093876,0.004032,0.168375,0.100969,0.055982,-0.109489,1.000000,0.043677,0.047152,0.174605,-0.023851,-0.152446,-0.116658,0.184992,-0.066013,0.102830,-0.071078,-0.035484,-0.103841,0.037093
7,-0.185457,0.072380,0.191442,-0.141302,0.156709,0.146327,0.043677,1.000000,-0.142786,-0.283411,-0.011480,0.047459,0.119721,0.165205,0.052546,0.140087,0.055292,-0.060218,0.034147,0.151551
8,-0.111444,0.108443,-0.139544,-0.080367,-0.105847,0.037683,0.047152,-0.142786,1.000000,0.013388,-0.096954,-0.015608,-0.089514,-0.145086,-0.060764,-0.090913,0.034089,0.045001,0.224811,-0.050685
9,0.067126,0.113275,-0.083372,0.006003,-0.027738,-0.033446,0.174605,-0.283411,0.013388,1.000000,-0.028911,0.020106,0.058519,-0.080598,-0.114921,-0.199393,0.002015,0.124195,0.046417,-0.045030


In [18]:
corr = np.delete(x_data_generated, [12,3],1)

In [20]:
corr_data = cross_val_score(RandomForestClassifier(), corr, y_data_generated, scoring='accuracy').mean()
corr_data

0.9199999999999999

In [21]:
selector = VarianceThreshold(threshold=1)
select_feach = selector.fit_transform(x_data_generated)
select_feach.shape

(100, 7)

In [22]:
var_threshold = cross_val_score(LogisticRegression(), select_feach, y_data_generated, scoring='accuracy').mean()
var_threshold

0.8800000000000001

In [23]:
select_kbest = SelectKBest(k=5).fit_transform(x_data_generated, y_data_generated)
select_kbest.shape

(100, 5)

In [24]:
kbest = cross_val_score(LogisticRegression(), select_kbest, y_data_generated, scoring='accuracy').mean()
kbest

0.89

In [25]:
selector_model = SelectFromModel(estimator=LogisticRegression(C=1, penalty='l2')).fit_transform(x_data_generated, y_data_generated)

In [26]:
select_from_model = cross_val_score(LogisticRegression(), selector_model, y_data_generated, scoring='accuracy').mean()
select_from_model

0.8700000000000001

In [27]:
columns = pd.DataFrame(x_data_generated).columns.tolist()
columns

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [28]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_data_generated,y_data_generated)
feature_imp = pd.Series(clf.feature_importances_,index=columns).sort_values(ascending=False)
col_feature_imp = list(feature_imp.index[:5])

In [29]:
feature_importance = x_data_generated[:, col_feature_imp]

In [30]:
random_forest = cross_val_score(LogisticRegression(), feature_importance, y_data_generated, scoring='accuracy').mean()
random_forest

0.89

In [31]:
sequential_feature = SequentialFeatureSelector(estimator=LogisticRegression(), n_features_to_select=7).fit_transform(x_data_generated, y_data_generated)

In [32]:
seq_feature_sel = cross_val_score(LogisticRegression(), sequential_feature, y_data_generated, scoring='accuracy').mean()
seq_feature_sel

0.93

In [33]:
temp = {'begin_data': [begin_data], 'corr': [corr_data], 'var_threshold': [var_threshold], 
     'kbest': [kbest], 'select_from_model': [select_from_model], 'random_forest': [random_forest],
    'seq_feature_sel': [seq_feature_sel] }
result_tab = pd.DataFrame(temp)
result_tab

,begin_data,corr,var_threshold,kbest,select_from_model,random_forest,seq_feature_sel
0,0.86,0.92,0.88,0.89,0.87,0.89,0.93


По итогам отбора признаков различными методами, наиболее значимые результаты показал метод SequentialFeatureSelector